구글 드라이브 마운트 & 라이브러리 import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pickle
import os

from rank_bm25 import BM25Okapi
import numpy as np
import pandas as pd
from mecab import MeCab
from tqdm import tqdm

In [6]:
origin_path = "/content/drive/MyDrive/활동/boaz/"
bert_test_csv_path = os.path.join(origin_path,"test_df.csv")
bm25_result_path = os.path.join(origin_path,"bm25_total")
bm25_train_parquet_path = os.path.join(origin_path,"bm25input_added_data.parquet")

BM25 학습시켰던 데이터셋 로드

In [7]:
bm25_train_df = pd.read_parquet(bm25_train_parquet_path)

In [8]:
bm25_train_df

,text,bm25_input
0,주문 상고 를 모두 기각 한다 상고 비용 은 원고 들 의 부담 으로 한다 이유 상고...,"[주문, 상고, 를, 모두, 기각, 한다, 상고, 비용, 은, 원고, 들, 의, 부..."
1,주문 상고 를 기각 한다 상고 비용 은 피고 의 부담 으로 한다 이유 상고 이유 를...,"[주문, 상고, 를, 기각, 한다, 상고, 비용, 은, 피고, 의, 부담, 으로, ..."
2,주문 원심 판결 을 파기 하 고 사건 을 서울 고등 법원 에 환송 한다 이유 1 피...,"[주문, 원심, 판결, 을, 파기, 하, 고, 사건, 을, 서울, 고등, 법원, 에..."
3,주문 피고인 의 항소 를 기각 한다 이유 1 항소 이유 의 요지 피고인 에 대한 원...,"[주문, 피고인, 의, 항소, 를, 기각, 한다, 이유, 1, 항소, 이유, 의, ..."
4,주문 상고 를 기각 한다 상고 소송비용 은 원고 의 부담 으로 한다 이유 원고 소송...,"[주문, 상고, 를, 기각, 한다, 상고, 소송비용, 은, 원고, 의, 부담, 으로..."
...,...,...
149951,주문 검사 의 항소 를 기각 한다 이유 1 공소사실 및 원심 의 판단 가 공소사실 ...,"[주문, 검사, 의, 항소, 를, 기각, 한다, 이유, 1, 공소사실, 및, 원심,..."
149952,주문 1 제 1 심판 결 을 취소 한다 2 원고 들 의 청구 를 모두 기각 한다 3...,"[주문, 1, 제, 1, 심판, 결, 을, 취소, 한다, 2, 원고, 들, 의, 청..."
149953,주문 원심 판결 중 임료 상당 손해 에 관한 피고 들 의 패소 부분 을 파기 하 고...,"[주문, 원심, 판결, 중, 임료, 상당, 손해, 에, 관한, 피고, 들, 의, 패..."
149954,주문 원심 판결 을 파기 하 고 사건 을 서울 고등 법원 에 환송 한다 이유 상고 ...,"[주문, 원심, 판결, 을, 파기, 하, 고, 사건, 을, 서울, 고등, 법원, 에..."


테스트 데이터셋 로드

In [9]:
test_df = pd.read_csv(bert_test_csv_path,index_col = 0)

In [10]:
test_df

,word_1,facts
0,가정폭력범죄의 처벌 등에 관한 특례법,"피고인은 피해자 B(여, 39세)와 2021. 3.경 이혼한 전 남편이다. 피고인은..."
1,형법,피고인은 2020. 8. 29.경 불상지에서 인터넷 'AP카페' 게시판에 접속하여 ...
2,형법,[범죄전력] 피고인은 2020. 9. 10. 울산지방법원에서 사기죄로 징역 8월을 ...
3,감염병의 예방 및 관리에 관한 법,"보건복지부장관, 시·도시자 또는 시장·군수·구청장은 감염병이 유행하면 감염병 전파를..."
4,감염병의 예방 및 관리에 관한 법,"보건복지부장관, 시·도시자 또는 시장·군수·구청장은 감염병이 유행하면 감염병 전파를..."
...,...,...
2132,형법,"피해자 B은 평택시 C에 있는 D를 운영하는 자이고, 피고인은 위 회사의 화물기사이..."
2133,형법,【범죄전력】 피고인은 2019. 7. 19. 의정부지방법원에서 사기죄로 징역 10월...
2134,형법,피고인은 피해자 B의 외조카로서 피해자 소유의 고양시 일산동구 C 소재 다가구주택 ...
2135,형법,[범죄전력] 피고인은 2021. 4. 22. 인천지방법원에서 컴퓨터등사용사기죄 등으...


테스트 데이터셋 형태소 분리

In [11]:
mecab = MeCab()

In [12]:
test_df["tokenized_facts"] = test_df["facts"].apply(lambda text: " ".join(mecab.morphs(text)) )
test_df["tokenized_facts"] = test_df["tokenized_facts"].apply(lambda text: text.split(" "))

In [13]:
test_df

,word_1,facts,tokenized_facts
0,가정폭력범죄의 처벌 등에 관한 특례법,"피고인은 피해자 B(여, 39세)와 2021. 3.경 이혼한 전 남편이다. 피고인은...","[피고인, 은, 피해자, B, (, 여, ,, 39, 세, ), 와, 2021, ...."
1,형법,피고인은 2020. 8. 29.경 불상지에서 인터넷 'AP카페' 게시판에 접속하여 ...,"[피고인, 은, 2020, ., 8, ., 29, ., 경, 불, 상지, 에서, 인..."
2,형법,[범죄전력] 피고인은 2020. 9. 10. 울산지방법원에서 사기죄로 징역 8월을 ...,"[[, 범죄, 전력, ], 피고인, 은, 2020, ., 9, ., 10, ., 울..."
3,감염병의 예방 및 관리에 관한 법,"보건복지부장관, 시·도시자 또는 시장·군수·구청장은 감염병이 유행하면 감염병 전파를...","[보건복지부, 장관, ,, 시, ·, 도시, 자, 또는, 시장, ·, 군수, ·, ..."
4,감염병의 예방 및 관리에 관한 법,"보건복지부장관, 시·도시자 또는 시장·군수·구청장은 감염병이 유행하면 감염병 전파를...","[보건복지부, 장관, ,, 시, ·, 도시, 자, 또는, 시장, ·, 군수, ·, ..."
...,...,...,...
2132,형법,"피해자 B은 평택시 C에 있는 D를 운영하는 자이고, 피고인은 위 회사의 화물기사이...","[피해자, B, 은, 평택시, C, 에, 있, 는, D, 를, 운영, 하, 는, 자..."
2133,형법,【범죄전력】 피고인은 2019. 7. 19. 의정부지방법원에서 사기죄로 징역 10월...,"[【, 범죄, 전력, 】, 피고인, 은, 2019, ., 7, ., 19, ., 의..."
2134,형법,피고인은 피해자 B의 외조카로서 피해자 소유의 고양시 일산동구 C 소재 다가구주택 ...,"[피고인, 은, 피해자, B, 의, 외조카, 로서, 피해자, 소유, 의, 고양시, ..."
2135,형법,[범죄전력] 피고인은 2021. 4. 22. 인천지방법원에서 컴퓨터등사용사기죄 등으...,"[[, 범죄, 전력, ], 피고인, 은, 2021, ., 4, ., 22, ., 인..."


테스트용으로 20 개추출

In [21]:
random_test_df = test_df.sample(n=20, random_state=42)

BM25 결과 로드

In [14]:
with open(origin_path+'bm25_result', 'rb') as bm25result_file:
    bm25result = pickle.load(bm25result_file)

In [22]:
columns = ["word_1","facts", "similar_precedent"]  # 열 이름 리스트
bm25_result_df = pd.DataFrame(columns=columns)
for idx in tqdm(range(20)):
  tokenized_query = random_test_df.iloc[idx][2]
  result = bm25result.get_scores(tokenized_query)
  del tokenized_query
  max_index = np.argmax(result)
  del result
  bm25_result_df.loc[len(bm25_result_df)] = [random_test_df.iloc[idx][0],random_test_df.iloc[idx][1],bm25_train_df.iloc[max_index][0]]
bm25_result_df.to_csv(origin_path+'bm25_result.csv')

100%|██████████| 20/20 [06:53<00:00, 20.67s/it]


In [23]:
bm25_result_df

,word_1,facts,similar_precedent
0,전기통신공사업법,『2018고단430』 피고인과 B은 2017. 9.경 친구인 C을 통해 알게 된 피...,주문 피고인 을 징역 10 월 에 처한다 다만 이 판결 확정 일 부터 2 년 간 위...
1,형법,피고인은 2021. 3. 17. 16:06경 연천군 B에 있는 C정형외과병원 원무과...,주문 피고인 을 징역 2 년 및 벌금 30 만 원 에 처한다 피고인 이 위 벌금 을...
2,근로기준법,"피고인은 서울 금천구 B건물, C호에 있는 (주)D의 실경영자로서 상시 근로자 4명...",주문 피고인 을 징역 8 월 에 처한다 이 사건 공소 사실 중 별지 범죄 일람표 1...
3,관세법,1. 밀수입으로 인한 관세법위반 외국으로부터 물품을 수입하는 경우 해당 물품의 품명...,주문 피고인 들 을 각 징역 6 월 에 처한다 다만 이 판결 확정 일 로부터 각 2...
4,정보통신망 이용 촉진 및 정보보호 등에 관한 법,"피고인은 2019. 7. 23. 23:30경 대구 수성구 등지에서, 인터넷 B카페 ...",주문 피고인 을 벌금 300 만 원 에 처한다 피고인 이 위 벌금 을 납입 하 지 ...
5,관세법,피고인은 2019. 7. 29. 제주 B에 있는 피고인 운영의 ‘C’에서 부당하게 ...,주문 피고인 A 를 징역 10 월 에 피고인 주식회사 B 을 벌금 15 000 00...
6,형법,피고인은 2019. 2. 18.경 장소를 알 수 없는 곳에서 인터넷 사이트인 ‘B’...,주문 피고인 을 징역 8 월 에 처한다 다만 이 판결 확정 일 부터 2 년 간 위 ...
7,형법,피고인은 2019. 11. 20. 창원지방법원 진주지원에서 특수협박죄 등으로 징역 ...,주문 피고인 을 징역 1 년 4 개월 및 벌금 80 만 원 에 처한다 피고인 이 위...
8,형법,"피고인은 피해자 B(가명, 여, 39세)와 모르는 사이이다. 피고인은 2020. 1...",주문 피고인 을 벌금 500 만 원 에 처한다 피고인 이 위 벌금 을 납입 하 지 ...
9,도로교통법,피고인은 (차량번호 1 생략) 아반떼 승용차를 운전하는 사람이다. 피고인은 2019...,주문 피고인 을 벌금 600 만 원 에 처한다 피고인 이 위 벌금 을 납입 하 지 ...
